In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import cProfile

In [ ]:
import copy

In [ ]:
import autoreload
import evotsc
autoreload.reload(evotsc)

In [ ]:
# Population-level constants
intergene = 1000
interaction_dist = 2500

In [ ]:
def plot_expr(indiv, plot_title, plot_name):
    
    temporal_expr, fitness = indiv.evaluate()
    
    nb_genes, nb_steps = temporal_expr.shape
    
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    
    plt.figure(figsize=(9, 6), dpi=200)
    
    plt.ylim(-0.1, 2.1)
    
    for gene in range(nb_genes):
        linestyle = 'solid' if indiv.genes[gene].orientation == 0 else 'dashed'
        plt.plot(temporal_expr[indiv.genes[gene].id, :],
                 linestyle=linestyle,
                 color=colors[indiv.genes[gene].id],
                 label=f'Gene {indiv.genes[gene].id}')
    
    plt.grid(linestyle=':')
    plt.xlabel('Time', fontsize='large')
    plt.ylabel('Expression level', fontsize='large')
    
    plt.legend(loc='center right')
    plt.title(plot_title + f' fitness: {fitness:.5}')
        
    plt.savefig(plot_name, dpi=300, bbox_inches='tight')

In [ ]:
def switch_1_gene(individual): # Evaluate all the individuals obtained by switching 1 gene of the given individual
    for i_switched_gene in range(len(genes)):
        # Switch the current gene 
        to_switch = individual.genes[i_switched_gene]
        to_switch.orientation = 1 - to_switch.orientation

        # Run the switched system and plot
        switch_res, switch_fit = individual.evaluate()
        print(f'Switched gene {i_switched_gene}, fitness: {switch_fit}')
        plot_expr(switch_res, switch_fit, f'switched_{i_switched_gene}')

        # Reset the gene to its former orientation
        to_switch.orientation = 1 - to_switch.orientation

In [ ]:
def linear_activation(arr):
    return np.maximum(0.0, np.minimum(arr, 2.0))

In [ ]:
def tanh_activation(beta, arr):
    return np.tanh(arr - 1.0 + beta) + 1.0

In [ ]:
def tanh_2x_activation(beta, arr):
    return np.tanh(2*(arr - 1.0) + beta) + 1.0

In [ ]:
init_genes = evotsc.Gene.generate(intergene=intergene, nb_genes=10)

In [ ]:
init_indiv = evotsc.Individual(genes=init_genes,
                               interaction_dist=interaction_dist,
                               interaction_coef=0.3,
                               nb_eval_steps=51,
                               activation=lambda arr: tanh_2x_activation(0.0, arr))

In [ ]:
mutation = evotsc.Mutation(intergene_mutation_prob=1e-2,
                           intergene_mutation_var=50,
                           inversion_prob=1e-3)

In [ ]:
population = evotsc.Population(init_indiv=init_indiv, nb_indivs=1000, mutation=mutation)

In [ ]:
inversion_only = evotsc.Mutation(intergene_mutation_prob=0.0,
                                 intergene_mutation_var=0.0,
                                 inversion_prob=1.0)

In [ ]:
test_genes = evotsc.Gene.generate(intergene=intergene, nb_genes=10)

In [ ]:
for i_gene, gene in enumerate(test_genes):
    gene.intergene += 100 * i_gene

In [ ]:
test_genes

In [ ]:
test_indiv = evotsc.Individual(genes=test_genes,
                               interaction_dist=interaction_dist,
                               interaction_coef=0.3,
                               nb_eval_steps=51,
                               activation=linear_activation)

In [ ]:
real_indiv = test_indiv.clone()

In [ ]:
plot_expr(real_indiv, plot_title='individual', plot_name='individual 0')

In [ ]:
for i in range(1, 11):
    real_indiv.mutate(inversion_only)
    plot_expr(real_indiv, plot_title='individual', plot_name=f'individual {i}')
    print(real_indiv)

In [ ]:
real_indiv.generate_inversion(mutation=inversion_only)

In [ ]:
real_indiv